In [ ]:
%env ALL_PROXY=http://127.0.0.1:33001
%env HTTP_PROXY=http://127.0.0.1:33001
%env HTTPS_PROXY=http://127.0.0.1:33001
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

In [ ]:
!curl google.com

# Import

If change the model, in training step, you need to 
1. change the log chapter
2. change the model code
3. change the wandb chapter

In [31]:
from tqdm import tqdm
import cv2
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import torch.nn as nn
from sklearn.model_selection import KFold, train_test_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp
import wandb

In [ ]:
import sys
sys.path.insert(0, "/root/Soil-Column-Procedures")

from API_functions.DL import load_data, log, seed, evaluate, precheck
from API_functions import file_batch as fb

# Hyperparameter and log

In [ ]:
my_parameters = {
    'seed': 409,

    'Kfold': None,
    'ratio': 0.2,

    'model': 'DeepLabv3+',   # model = 'U-Net', 'DeepLabv3+', 'PSPNet'
    'optimizer': 'adam',
    'learning_rate':  0.0001,
    'batch_size': 32,
    'loss_function': 'cross_entropy',

    'n_epochs': 1000,
    'patience': 50,
}

device = 'cuda'
mylogger = log.Logger('all')

seed.stablize_seed(my_parameters['seed'])

# Transform

In [ ]:
# For training data
transform_train = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=90, p=0.5),
    A.GaussNoise(p=0.5),
    # A.OpticalDistortion(p=0.5),
    ToTensorV2(),
], seed=my_parameters['seed'])

# For validation and test data
transform_val = A.Compose([
    ToTensorV2(),
], seed=my_parameters['seed'])

# Model

In [ ]:
# model = smp.PSPNet(
#     encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,                      # model output channels (number of classes in your dataset)
# )
# model = model.to(device)

In [ ]:
# model = smp.Unet(
#     encoder_name="efficientnet-b0",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,                      # model output channels (number of classes in your dataset)
# )
# model = model.to(device)

In [ ]:
model = smp.DeepLabV3Plus(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=my_parameters['learning_rate'])
criterion = evaluate.DiceBCELoss()

# Train

The codes below are only for training.

In test step, you need to proceed the codes above and the test chapter code.

## Wandb

In [ ]:
wandb.init(
    project="U-Net",
    name='13.DeepLab, add noisy data',
    config=my_parameters,
)

## Load_data

In [ ]:
data_paths = fb.get_image_names('/root/Soil-Column-Procedures/data/version0/train_images/', None, 'png')
labels_paths = fb.get_image_names('/root/Soil-Column-Procedures/data/version0/train_labels/', None, 'png')


data = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in data_paths]
labels = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in labels_paths]
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=my_parameters['ratio'], random_state=my_parameters['seed'])


train_dataset = load_data.my_Dataset(train_data, train_labels, transform=transform_train)
val_dataset = load_data.my_Dataset(val_data, val_labels, transform=transform_val)


train_loader = DataLoader(train_dataset, batch_size=my_parameters['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=my_parameters['batch_size'], shuffle=False)


print(f'len of train_data: {len(train_data)}, len of val_data: {len(val_data)}')

## Train

In [ ]:
val_loss_best = 100000
proceed_once = True  # Add a flag

for epoch in range(my_parameters['n_epochs']):
    model.train()
    train_loss = 0.0

    for images, labels in tqdm(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)

        # Checking the dimension of the outputs and labels
        if outputs.dim() == 4 and outputs.size(1) == 1:
            outputs = outputs.squeeze(1)
        
        # Only proceed once:
        if proceed_once:
            print(f'outputs.size(): {outputs.size()}, labels.size(): {labels.size()}')
            print(f'outputs.min: {outputs.min()}, outputs.max: {outputs.max()}')
            print(f'labels.min: {labels.min()}, labels.max: {labels.max()}')
            print(f'count of label 0: {(labels == 0).sum()}, count of label 1:{(labels == 1).sum()}')
            print('')
            proceed_once = False  # Set the flag to False after proceeding once
        
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
    
    train_loss_mean = train_loss / len(train_loader.dataset)


    model.eval()
    val_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            if outputs.dim() == 4 and outputs.size(1) == 1:
                outputs = outputs.squeeze(1)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)

    val_loss_mean = val_loss / len(val_loader.dataset)
    dict = {'train_loss': train_loss_mean, 'epoch': epoch, 'val_loss': val_loss_mean}
    mylogger.log(dict)

    if val_loss_mean < val_loss_best:
        val_loss_best = val_loss_mean
        torch.save(model.state_dict(), f"model_{my_parameters['model']}.pth")
        print(f'Model saved at epoch {epoch}, val_loss: {val_loss_mean}')

In [ ]:
wandb.finish()

# Test

If changes the model
1. Test inference6, 7, 8 are for U-Net, DeeplabV3+, PSPNet. Remember to change the path.
2. Do change the 'model' chapter in train step.

If changes the data
1. Test inference path
2. Dataloader

## Test model

In [32]:
def test_model(model, test_loader, test_names, device='cuda'):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Turn off gradients to speed up this part
        loss = []
        dice = []
        soft_dice = []
        bce_loss = []
        iou = []
        f1_score = []

        proceed_once = True  # Add a flag

        for i, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)

            # Checking the dimension of the outputs and labels
            if outputs.dim() == 4 and outputs.size(1) == 1:
                outputs = outputs.squeeze(1)

            # Only proceed once:
            if proceed_once:
                print(f'outputs.size(): {outputs.size()}, labels.size(): {labels.size()}')
                print(f'outputs.min: {outputs.min()}, outputs.max: {outputs.max()}')
                print(f'labels.min: {labels.min()}, labels.max: {labels.max()}')
                print(f'count of label 0: {(labels == 0).sum()}, count of label 1:{(labels == 1).sum()}')
                print('')
                proceed_once = False

            # Calculate loss indexes 
            # 1.bce and dice loss, because the criterion function is used in train process, so it has a sigmoid function inside, should be put before sigmoid
            loss.append(criterion(outputs, labels).item())

            # 2.Calculate dice, soft_dice, bce_loss 
            outputs = torch.sigmoid(outputs)  # Apply sigmoid to get values between 0 and 1

            dice.append(evaluate.dice_coefficient(outputs, labels))
            soft_dice.append(evaluate.soft_dice_coefficient(labels, outputs))
            calculate_bce = nn.BCELoss()
            bce_loss.append(calculate_bce(outputs, labels))
            iou.append(evaluate.iou(pred=outputs, target=labels, n_classes=2))
            f1_score.append(evaluate.f1_score(pred=outputs, gt=labels))

            # Save output images
            outputs = outputs > 0.5  # Threshold the probabilities to create a binary mask
            for j, img in enumerate(outputs):
                save_path = f'/root/Soil-Column-Procedures/data/version1/inference/1/'
                save_path = save_path + test_names[j]
                output_np = img.cpu().numpy().astype(np.uint8) * 255  # Convert to numpy array and scale to 0-255
                cv2.imwrite(save_path, output_np)

            print(f'Processed batch {i+1}/{len(test_loader)}')
        
        loss_avg = sum(loss) / len(test_loader)
        dice_avg = sum(dice) / len(test_loader)
        soft_dice_avg = sum(soft_dice) / len(test_loader)
        bce_loss_avg = sum(bce_loss) / len(test_loader)
        iou_avg = sum(iou) / len(test_loader)
        f1_score_avg = sum(f1_score) / len(test_loader)
        print(f'Loss: {loss_avg}, Dice: {dice_avg}, soft_dice: {soft_dice_avg}, BCE Loss: {bce_loss_avg}, IOU: {iou_avg}, f1_score: {f1_score_avg}')


In [33]:
# Test the model, using the model define before

model.load_state_dict(torch.load(f"model_{my_parameters['model']}.pth", weights_only=True))

model = model.to(device)

## Test data loader

In [34]:
test_paths = fb.get_image_names('/root/Soil-Column-Procedures/data/version1/test_images/', None, 'png')
test_labels_paths = fb.get_image_names('/root/Soil-Column-Procedures/data/version1/test_labels/', None, 'png')
tests = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in test_paths]
test_labels = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) for p in test_labels_paths]

tests, test_labels = precheck.precheck(tests, test_labels)

test_dataset = load_data.my_Dataset(tests, test_labels, transform=transform_val)
test_loader = DataLoader(test_dataset, batch_size=my_parameters['batch_size'], shuffle=False)

print(f'len of test_data: {len(tests)}')

test_names = [p.split('/')[-1] for p in test_paths]

1 images have been found in /root/Soil-Column-Procedures/data/version1/test_images/
All images are:
/root/Soil-Column-Procedures/data/version1/test_images/0028.385.png
Get names completely!
1 images have been found in /root/Soil-Column-Procedures/data/version1/test_labels/
All images are:
/root/Soil-Column-Procedures/data/version1/test_labels/0028.385.png
Get names completely!
Check 1 pass, Dataset and labels have the same shape
2.Dataset images have been padded
2.Label images have been padded
Check 3 pass, Images and labels are 8-bit
Check 4 pass, Labels contain only 0 and 255
len of test_data: 1


## Test

In [35]:
test_model(model, test_loader, test_names)

outputs.size(): torch.Size([1, 800, 800]), labels.size(): torch.Size([1, 800, 800])
outputs.min: -10.441927909851074, outputs.max: 26.676090240478516
labels.min: 0.0, labels.max: 1.0
count of label 0: 635364, count of label 1:4636

Processed batch 1/1
Loss: 0.0572110153734684, Dice: 0.8596512675285339, soft_dice: 0.8950426578521729, BCE Loss: 0.0094646867364645, IOU: 0.7538494439692045, f1_score: 0.8596512620412144


/root/Soil-Column-Procedures/API_functions/DL/load_data.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img = torch.tensor(img, dtype=torch.float32)
/root/Soil-Column-Procedures/API_functions/DL/load_data.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label/255, dtype=torch.float32)


# Visualize

In [ ]:
from API_functions.Visual import file_compare as fc
# %matplotlib qt

db = fc.ImageDatabase()
# image_processor.add_result('pre_processed', tpi.user_threshold(image_processor.image, 160))
zoom = fc.ZoomRegion(200, 400, 100, 200)
db.add_additional_folder('/root/Soil-Column-Procedures/data/version1/test_images/', 'test_images')
db.add_additional_folder('/root/Soil-Column-Procedures/data/version1/test_labels/', 'test_labels')
db.add_additional_folder('/root/Soil-Column-Procedures/data/version1/inference/1/', 'test_inference')
image_processor = db.get_image_processor('0028.386.png')
image_processor.show_images('test_images', 'test_inference', 'test_labels', zoom_region=zoom)